<a href="https://colab.research.google.com/github/RafaelTorresCH/Signal-theory/blob/main/class%20notebooks/NUEVA_BASE_DE_DATOS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#https://www.kaggle.com/datasets/amirberenji/electric-motor-vibrations-dataset?resource=download

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("amirberenji/electric-motor-vibrations-dataset")

print("Path to dataset files:", path)

100%|██████████| 38.4M/38.4M [00:00<00:00, 78.7MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/amirberenji/electric-motor-vibrations-dataset/versions/1
